In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from qgrid import show_grid as Grid

In [2]:
loans=pd.read_csv('lending-club-data.csv')
loans.head()

C:\Users\Keulando\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [4]:
loans.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

In [5]:
loans['safe_loans']=loans['bad_loans'].apply(lambda x: +1 if x==0 else -1)
loans.drop('bad_loans', axis=1, inplace=True)

In [6]:
perc_1=loans['safe_loans'].value_counts().iloc[0]/len(loans['safe_loans'])

In [7]:
perc_1

0.81118533199572618

In [8]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[ [target] +features ]

In [9]:
# one hot encode features
loans=pd.get_dummies(loans)

In [10]:
loans.head()


,safe_loans,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,grade_A,grade_B,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,1,0,11,27.65,1,1,83.7,0.00,0,1,...,0,0,0,0,0,0,0,0,1,0
1,-1,1,1,1.00,1,1,9.4,0.00,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,11,8.72,1,1,98.5,0.00,0,0,...,0,0,0,0,0,1,0,0,1,0
3,1,0,11,20.00,0,1,21.0,16.97,0,0,...,0,0,0,0,1,0,0,0,1,0
4,1,0,4,11.20,1,1,28.3,0.00,1,0,...,0,0,0,0,0,0,0,1,1,0


In [76]:
train_index=pd.read_json('module-5-assignment-1-train-idx.json', typ='series')
validation_idx= pd.read_json('module-5-assignment-1-validation-idx.json', typ='series')
train=loans.iloc[train_index]
validation=loans.iloc[validation_idx]

In [77]:
len(train), len(validation)

(37224, 9284)

In [78]:
# build clf

In [79]:
X=train.iloc[:,1:]
y=train.iloc[:,0]
decision_tree_model=DecisionTreeClassifier(max_depth=6).fit(X,y)
small_model=DecisionTreeClassifier(max_depth=2).fit(X,y)


In [80]:
# visualize it

In [16]:
def plot_mushroom_boundary(X, y, fitted_model):

    plt.figure(figsize=(9.8,5), dpi=100)
    
    for i, plot_type in enumerate(['Decision Boundary', 'Decision Probabilities']):
        plt.subplot(1,2,i+1)

        mesh_step_size = 0.01  # step size in the mesh
        x_min, x_max = X.iloc[:, 0].min() - .1, X.iloc[:, 0].max() + .1
        y_min, y_max = X.iloc[:, 1].min() - .1, X.iloc[:, 1].max() + .1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step_size), np.arange(y_min, y_max, mesh_step_size))
        if i == 0:
            Z = fitted_model.predict(np.c_[xx, yy])
        else:
            try:
                Z = fitted_model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
            except:
                plt.text(0.4, 0.5, 'Probabilities Unavailable', horizontalalignment='center',
                     verticalalignment='center', transform = plt.gca().transAxes, fontsize=12)
                plt.axis('off')
                break
        Z = Z.reshape(xx.shape)
        plt.scatter(X[y.values==0,0], X[y.values==0,1], alpha=0.4, label='Edible', s=5)
        plt.scatter(X[y.values==1,0], X[y.values==1,1], alpha=0.4, label='Posionous', s=5)
        plt.imshow(Z, interpolation='nearest', cmap='RdYlBu_r', alpha=0.15, 
                   extent=(x_min, x_max, y_min, y_max), origin='lower')
        plt.title(plot_type + '\n' + 
                  str(fitted_model).split('(')[0]+ ' Test Accuracy: ' + str(np.round(fitted_model.score(X, y), 5)))
        plt.gca().set_aspect('equal');
        
    plt.tight_layout()
    plt.subplots_adjust(top=0.9, bottom=0.08, wspace=0.02)

In [16]:
import matplotlib.pyplot as plt
%matplotlib notebook
# x needs to have same num of feaatures as model depth
plot_mushroom_boundary(X,y,small_model)

<IPython.core.display.Javascript object>

ValueError: Number of features of the model must match the input. Model n_features is 67 and input n_features is 242 

In [81]:
#consider 2 pos adn 2 neg examples
validation_safe_loans=validation[validation[target]==1]
validation_risky_loans=validation[validation[target]==-1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

,safe_loans,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,grade_A,grade_B,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
19,1,0,11,11.18,1,1,82.4,0.0,0,1,...,0,0,0,0,0,0,0,0,1,0
79,1,0,10,16.85,1,1,96.4,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
24,-1,0,3,13.97,0,1,59.5,0.0,0,0,...,0,0,0,0,1,0,0,0,0,1
41,-1,0,11,16.33,1,1,62.1,0.0,1,0,...,0,0,0,0,0,0,0,0,1,0


# Q1  What percentage of the predictions on sample_validation_data did decision_tree_model get correct?

In [82]:
decision_tree_model.predict(sample_validation_data.iloc[:,1:])

array([ 1, -1, -1,  1], dtype=int64)

In [83]:
(sample_validation_data['safe_loans'] == decision_tree_model.predict(sample_validation_data.iloc[:,1:])).sum()/float(len(sample_validation_data))


0.5

Checkpoint: Can you verify that for all the predictions with probability >= 0.5, the model predicted the label +1

# Q2 Which loan has the highest probability of being classified as a safe loan?



In [84]:
proba=decision_tree_model.predict_proba(sample_validation_data.iloc[:,1:])
proba

array([[ 0.34156543,  0.65843457],
       [ 0.53630646,  0.46369354],
       [ 0.64750958,  0.35249042],
       [ 0.20789474,  0.79210526]])

In [85]:
for x,j in proba:

    if x<0.5 and j>=0.5:
        print(x,j)

0.341565425214 0.658434574786
0.207894736842 0.792105263158


# Q3 Based on the visualized tree, what prediction would you make for this data point (according to small_model)? (If you don't have Graphviz, you can answer this quiz question by executing the next part.)

In [86]:
from sklearn.tree import export_graphviz as gviz


with open('tree_model_small','w') as f:
    f=gviz(small_model, out_file=f, feature_names=X.columns)
    

# Evaluating accuracy 


In [89]:
big_model=DecisionTreeClassifier(max_depth=10).fit(X,y)

accu_big_train=big_model.score(X,y)
accu_big_valid=big_model.score(X_valid,y_valid)

accu_big_train, accu_big_valid


(0.6637384483129164, 0.62688496337785438)

In [90]:
from sklearn.metrics import accuracy_score

X_train=train.iloc[:,1:]
y_train=train.iloc[:,0]

X_valid=validation.iloc[:,1:]
y_valid=validation.iloc[:,0]

small_model_predict_train=small_model.predict(X_train)
decision_tree_predict_train=decision_tree_model.predict(X_train)
big_model_predict_train=big_model.predict(X_train)

small_model_predict_valid=small_model.predict(X_valid)
decision_tree_predict_valid=decision_tree_model.predict(X_valid)
big_model_predict_valid=big_model.predict(X_valid)

print('Accuracy on train data')
print('Accuracy for small: {:.2f}\n Accuracy for decision_model: {:.2f}\n Accuracy for big: {:.2f}'.format(accuracy_score(y_train, small_model_predict_train),
                                                                                                         accuracy_score(y_train, decision_tree_predict_train),
                                                                                                          accuracy_score(y_train, big_model_predict_train)))

print('-'*10)
print('Accuracy on validation data')
print('Accuracy for small: {:.2f}\n Accuracy for decision_model: {:.2f}\n Accuracy for big: {:.2f}'.format(accuracy_score(y_valid, small_model_predict_valid),
                                                                                                         accuracy_score(y_valid, decision_tree_predict_valid),
                                                                                                          accuracy_score(y_valid, big_model_predict_valid)))

Accuracy on train data
Accuracy for small: 0.61
 Accuracy for decision_model: 0.64
 Accuracy for big: 0.66
----------
Accuracy on validation data
Accuracy for small: 0.62
 Accuracy for decision_model: 0.64
 Accuracy for big: 0.63


## Q6 How does the performance of big_model on the validation set compare to decision_tree_model on the validation set? Is this a sign of overfitting?